In [6]:
# Importing relevant libraries
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer


In [1]:
# Configuruing path to kaggle json file
import json
import os

# Ensuring the folder for Kaggle exists
os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)

# Moving the kaggle.json file to the ~/.kaggle/ directory
with open('kaggle.json', 'r') as f:
    creds = json.load(f)

# Creating the kaggle.json in the ~/.kaggle directory
with open(os.path.expanduser('~/.kaggle/kaggle.json'), 'w') as f:
    json.dump(creds, f)

# Setting the appropriate file permissions
os.chmod(os.path.expanduser('~/.kaggle/kaggle.json'), 0o600)


In [2]:
# Using API to fetch dataset from Kaggle
! kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 98% 79.0M/80.9M [00:05<00:00, 20.9MB/s]
100% 80.9M/80.9M [00:05<00:00, 15.4MB/s]


In [3]:
# Extracting the compressed dataset
import zipfile

with zipfile.ZipFile('sentiment140.zip', 'r') as zip_ref:
    zip_ref.extractall()
    print("The dataset is extracted")

The dataset is extracted


In [4]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
# Printing the stopwords in English
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data processing

In [12]:
# Loading the data
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1")


In [13]:
data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [14]:
# Checking number of rows and columns
data.shape

(1599999, 6)

In [17]:
# First datapoint is taken as column names, so creating new list for original column names
data.columns=["target","ids","date","flag", "user","text"]

# Loading data again
data = pd.read_csv("training.1600000.processed.noemoticon.csv",names=data.columns, encoding = "ISO-8859-1")


In [18]:
data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [19]:
data.shape

(1600000, 6)

In [20]:
# Finding missing values
data.isna().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [21]:
# No missing values

In [22]:
# Checking distributuion of target column
data.target.value_counts()

,count
target,
0,800000
4,800000


In [23]:
# Changing label "4" to "1"
data['target'].replace(4, 1, inplace=True)

<ipython-input-23-220d99b8d958>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['target'].replace(4, 1, inplace=True)


In [24]:
data.target.value_counts()

,count
target,
0,800000
1,800000


# Stemming

In [25]:
port_stem=PorterStemmer()

In [26]:
# Defining a function to reduce each word to it's root word
def stemming(content):
    stemmed_content=re.sub("[^a-zA-Z]", " ", content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words("english")]
    stemmed_content = " ".join(stemmed_content)
    return stemmed_content


In [27]:
# Applying the function to the text column of the dataset
data["stemmed_text"]=data["text"].apply(stemming)

In [28]:
data.head()

,target,ids,date,flag,user,text,stemmed_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [29]:
print(data.stemmed_text)

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_text, Length: 1600000, dtype: object


# Training the model

In [30]:
# Seperating features and label
features = data["stemmed_text"].values
label = data["target"].values

In [31]:
# Splitting the data for training
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(features,label,test_size=0.2, random_state = 2, stratify = label)

In [32]:
# Converting the textual data to numerical data for computation by the model
vectorizer=TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


In [33]:
print(X_train)

  (0, 436713)	0.2725987626483838
  (0, 354543)	0.3588091611460021
  (0, 185193)	0.5277679060576009
  (0, 109306)	0.3753708587402299
  (0, 235045)	0.41996827700291095
  (0, 443066)	0.4484755317023172
  (1, 160636)	1.0
  (2, 109306)	0.4591176413728317
  (2, 124484)	0.1892155960801415
  (2, 407301)	0.1870933868497303
  (2, 129411)	0.29074192727957143
  (2, 406399)	0.32105459490875526
  (2, 433560)	0.3296595898028565
  (2, 77929)	0.31284080750346344
  (2, 443430)	0.3348599670252845
  (2, 266729)	0.24123230668976975
  (2, 409143)	0.15169282335109835
  (2, 178061)	0.1619010109445149
  (2, 150715)	0.18803850583207948
  (2, 132311)	0.20289715703997938
  (2, 288470)	0.16786949597862733
  (3, 406399)	0.29029991238662284
  (3, 158711)	0.44569393722995737
  (3, 151770)	0.278559647704793
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 318303)	0.21254698865277746
  (1279996, 434014)	0.2718945052332447
  (1279996, 390130)	0.2206474219107611
  (1279996, 373144)	0.3521250099983204
  (1279996, 238077)

In [35]:
# Since this is a binary classification problem, use LogisticRegression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter = 1000)
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

# Evaluation of the model

In [37]:
# Accuracy score for training data
from sklearn.metrics import accuracy_score
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train,X_train_prediction)
print("Accuracy score of training data:", training_data_accuracy)

Accuracy score of training data: 0.79871953125


In [38]:
# Accuracy score for testing data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test,X_test_prediction)
print("Accuracy score of testing data:", test_data_accuracy)

Accuracy score of testing data: 0.77668125


# Saving the trained model

In [44]:
import pickle

In [45]:
filename= "twitter_sentiment_analysis_model.sav"
pickle.dump(model,open(filename,"wb"))

# Using the saved model for predictions

In [46]:
# loading the saved model
loaded_model = pickle.load(open("twitter_sentiment_analysis_model.sav", "rb"))

In [53]:
X_new = X_test[200]

In [54]:
print(Y_test[200])

1


In [55]:
prediction=model.predict(X_new)
print(prediction)

[1]


In [56]:
X_new2= X_test[1000]

In [57]:
print(Y_test[1000])

0


In [58]:
prediction2=model.predict(X_new2)
print(prediction2)

[0]
